In [1]:
import numpy as np

from edgedroid.models.timings import *
import edgedroid.data as default_data

exec_time_data, neuroticism_data, imp_data, dur_data = default_data.load_default_exec_time_data()
exec_time_data

In [2]:
data = preprocess_data(exec_time_data, neuroticism_data, imp_data, dur_data)
data

# Validation

For each `run_id`, we
1. Build a model excluding the data associated to that `run_id`.
2. Execute X runs of the model following the sequence of TTFs in the excluded run.
3. Collect the TTFs generated per step by the model.
4. Calculate error w.r.t. excluded `run_id`.


In [3]:
from collections import deque
import pandas as pd
from typing import Tuple

result_rows = deque()

for run_id in data["run_id"].unique():
    test_data = data[data["run_id"] == run_id].copy()
    
    def get_test_data(*args, **kwargs) -> Tuple[pd.DataFrame, pd.arrays.IntervalArray, pd.arrays.IntervalArray, pd.arrays.IntervalArray]:
        etime_data = exec_time_data[exec_time_data["run_id"] != run_id].copy()
        return etime_data, neuroticism_data, imp_data, dur_data
    
    class EmpiricalTestModel(EmpiricalETM):
        get_data = get_test_data
        
    class NaiveTestModel(FirstOrderETM):
        get_data = get_test_data
    
    neuroticism = test_data["neuroticism_raw"].unique()[0]
    empirical_model = EmpiricalTestModel(neuroticism=neuroticism, ttf_levels=4, winsorize=True)
    naive_model = NaiveTestModel()
    
    for _ in range(30):
        for i, (prev_ttf, exec_time) in enumerate(test_data[["ttf", "next_exec_time"]].itertuples(index=False)):
            if i != 0:
                # first row has no previous ttf
                empirical_model.advance(prev_ttf)
                naive_model.advance(prev_ttf)
                
            predicted_realistic_exec_time = empirical_model.get_execution_time()
            predicted_naive_exec_time = naive_model.get_execution_time()
            result_rows.append({
                "real": exec_time, 
                "prediction": predicted_realistic_exec_time, 
                "model": "realistic"
            })
            result_rows.append({
                "real": exec_time,
                "prediction": predicted_naive_exec_time,
                "model": "1st order"
            })
            
results = pd.DataFrame(result_rows)
results

In [4]:
import numpy as np

results["error"] = results["prediction"] - results["real"]
results["sqr_error"] = np.square(results["error"])

results

In [5]:
results.drop(columns=["error"]).groupby("model").describe().drop(columns=["real", "prediction"]).T

In [6]:
# %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

ax = sns.histplot(data=results, x="sqr_error", hue="model", stat="density", log_scale=True, cumulative=True)
ax.set_title("CDF of Model Error")
ax.set_xlabel("Error (% w.r.t. test value)")
plt.show()